# Ensemble Learning

## Initial Imports

In [28]:
import warnings
warnings.filterwarnings('ignore')

In [29]:
import numpy as np
import pandas as pd
from pathlib import Path
from collections import Counter

In [30]:
from sklearn.metrics import balanced_accuracy_score
from sklearn.metrics import confusion_matrix
from imblearn.metrics import classification_report_imbalanced
from sklearn.preprocessing import LabelEncoder
from sklearn.ensemble import RandomForestClassifier
from sklearn.preprocessing import StandardScaler


## Read the CSV and Perform Basic Data Cleaning

In [31]:
# Load the data
file_path = Path('LoanStats_2019Q1.csv')
df = pd.read_csv(file_path)

df_binary_encoded = pd.get_dummies(df, columns=["home_ownership", 'verification_status'])
df_binary_encoded

le = LabelEncoder()
le.fit(df_binary_encoded["issue_d"])
df_binary_encoded["issue_d"] = le.transform(df_binary_encoded["issue_d"])
le.fit(df_binary_encoded["loan_status"])
df_binary_encoded["loan_status"] = le.transform(df_binary_encoded["loan_status"])
le.fit(df_binary_encoded["hardship_flag"])
df_binary_encoded["hardship_flag"] = le.transform(df_binary_encoded["hardship_flag"])
le.fit(df_binary_encoded["debt_settlement_flag"])
df_binary_encoded["debt_settlement_flag"] = le.transform(df_binary_encoded["debt_settlement_flag"])
le.fit(df_binary_encoded["application_type"])
df_binary_encoded["application_type"] = le.transform(df_binary_encoded["application_type"])
le.fit(df_binary_encoded["initial_list_status"])
df_binary_encoded["initial_list_status"] = le.transform(df_binary_encoded["initial_list_status"])
le.fit(df_binary_encoded["next_pymnt_d"])
df_binary_encoded["next_pymnt_d"] = le.transform(df_binary_encoded["next_pymnt_d"])
le.fit(df_binary_encoded["pymnt_plan"])
df_binary_encoded["pymnt_plan"] = le.transform(df_binary_encoded["pymnt_plan"])

#df_binary_encoded['pymnt_plan'].value_counts()

## Split the Data into Training and Testing

In [32]:
#features and target

# Create our features
X = df_binary_encoded.copy()
X.drop('loan_status', axis=1)

# Create our target
y = df_binary_encoded['loan_status']

In [33]:
X.describe()

,loan_amnt,int_rate,installment,annual_inc,issue_d,loan_status,pymnt_plan,dti,delinq_2yrs,inq_last_6mths,...,total_il_high_credit_limit,hardship_flag,debt_settlement_flag,home_ownership_ANY,home_ownership_MORTGAGE,home_ownership_OWN,home_ownership_RENT,verification_status_Not Verified,verification_status_Source Verified,verification_status_Verified
count,68817.000000,68817.000000,68817.000000,6.881700e+04,68817.000000,68817.000000,68817.0,68817.000000,68817.000000,68817.000000,...,6.881700e+04,68817.0,68817.0,68817.000000,68817.000000,68817.000000,68817.000000,68817.000000,68817.000000,68817.000000
mean,16677.594562,0.127718,480.652863,8.821371e+04,0.805542,0.994958,0.0,21.778153,0.217766,0.497697,...,5.572240e+04,0.0,0.0,0.009285,0.526309,0.106747,0.357659,0.478007,0.373992,0.148001
std,10277.348590,0.048130,288.062432,1.155800e+05,0.714932,0.070831,0.0,20.199244,0.718367,0.758122,...,5.095845e+04,0.0,0.0,0.095914,0.499311,0.308793,0.479314,0.499520,0.483865,0.355104
min,1000.000000,0.060000,30.890000,4.000000e+01,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,...,1.270000e+02,0.0,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,9000.000000,0.088100,265.730000,5.000000e+04,0.000000,1.000000,0.0,13.890000,0.000000,0.000000,...,2.288000e+04,0.0,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
50%,15000.000000,0.118000,404.560000,7.300000e+04,1.000000,1.000000,0.0,19.760000,0.000000,0.000000,...,4.200000e+04,0.0,0.0,0.000000,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000
75%,24000.000000,0.155700,648.100000,1.040000e+05,1.000000,1.000000,0.0,26.660000,0.000000,1.000000,...,7.249900e+04,0.0,0.0,0.000000,1.000000,0.000000,1.000000,1.000000,1.000000,0.000000
max,40000.000000,0.308400,1676.230000,8.797500e+06,2.000000,1.000000,0.0,999.000000,18.000000,5.000000,...,1.426964e+06,0.0,0.0,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000


In [34]:
# Check the balance of our target values
y.value_counts()

1    68470
0      347
Name: loan_status, dtype: int64

In [35]:
# Split the X and y into X_train, X_test, y_train, y_test
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=1)

## Data Pre-Processing

Scale the training and testing data using the `StandardScaler` from `sklearn`. Remember that when scaling the data, you only scale the features data (`X_train` and `X_testing`).

In [36]:
# Create the StandardScaler instance
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()

In [37]:
# Fit the Standard Scaler with the training data
# When fitting scaling functions, only train on the training dataset
X_scaler = scaler.fit(X_train)

In [38]:
# Scale the training and testing data
X_train_scaled = X_scaler.transform(X_train)
X_test_scaled = X_scaler.transform(X_test)

## Ensemble Learners

In this section, you will compare two ensemble algorithms to determine which algorithm results in the best performance. You will train a Balanced Random Forest Classifier and an Easy Ensemble classifier . For each algorithm, be sure to complete the folliowing steps:

1. Train the model using the training data. 
2. Calculate the balanced accuracy score from sklearn.metrics.
3. Display the confusion matrix from sklearn.metrics.
4. Generate a classication report using the `imbalanced_classification_report` from imbalanced-learn.
5. For the Balanced Random Forest Classifier only, print the feature importance sorted in descending order (most important feature to least important) along with the feature score

Note: Use a random state of 1 for each algorithm to ensure consistency between tests

### Balanced Random Forest Classifier

In [39]:
# Resample the training data with the BalancedRandomForestClassifier
from imblearn.ensemble import BalancedRandomForestClassifier
model = BalancedRandomForestClassifier(n_estimators=1000, random_state=1)
model = model.fit(X_train_scaled, y_train)
#rf_model = rf_model.fit(X_train_scaled, y_train)

In [40]:
# Calculated the balanced accuracy score
y_pred = model.predict(X_test_scaled)
balanced_accuracy_score(y_test, y_pred)

0.9985383536014967

In [41]:
# Display the confusion matrix
confusion_matrix(y_test, y_pred)

array([[  101,     0],
       [   50, 17054]], dtype=int64)

In [42]:
# Print the imbalanced classification report
print(classification_report_imbalanced(y_test, y_pred))

                   pre       rec       spe        f1       geo       iba       sup

          0       0.67      1.00      1.00      0.80      1.00      1.00       101
          1       1.00      1.00      1.00      1.00      1.00      1.00     17104

avg / total       1.00      1.00      1.00      1.00      1.00      1.00     17205



In [43]:
# List the features sorted in descending order by feature importance
importances = model.feature_importances_
sorted(zip(model.feature_importances_, X.columns), reverse=True)


[(0.44291136687720256, 'loan_status'),
 (0.047761801151201974, 'total_rec_prncp'),
 (0.045679129052958414, 'last_pymnt_amnt'),
 (0.043395193539149925, 'total_pymnt'),
 (0.03930914028866674, 'total_rec_int'),
 (0.03741844765427699, 'total_pymnt_inv'),
 (0.01923338074254517, 'int_rate'),
 (0.010897833414539529, 'issue_d'),
 (0.010205122438934568, 'mths_since_recent_inq'),
 (0.008677183593277448, 'dti'),
 (0.008258797128342566, 'installment'),
 (0.008171126375351572, 'max_bal_bc'),
 (0.008114901876564, 'out_prncp'),
 (0.008106152128692198, 'out_prncp_inv'),
 (0.007714809002994088, 'total_bc_limit'),
 (0.007551186165715264, 'annual_inc'),
 (0.007438607323447561, 'revol_bal'),
 (0.007362051991227451, 'tot_hi_cred_lim'),
 (0.007166167683468682, 'total_bal_il'),
 (0.00706892633348977, 'mths_since_rcnt_il'),
 (0.006967583565897205, 'total_rev_hi_lim'),
 (0.006950271368611943, 'bc_open_to_buy'),
 (0.006836597551744597, 'mo_sin_old_rev_tl_op'),
 (0.006540888334620546, 'il_util'),
 (0.00643710805

### Easy Ensemble Classifier

In [44]:
# Train the Classifier
model = RandomForestClassifier(n_estimators=1000, random_state=0)
model = model.fit(X_train_scaled, y_train)


In [45]:
# Calculated the balanced accuracy score
y_pred = model.predict(X_test_scaled)
balanced_accuracy_score(y_test, y_pred)

1.0

In [46]:
# Display the confusion matrix
confusion_matrix(y_test, y_pred)

array([[  101,     0],
       [    0, 17104]], dtype=int64)

In [47]:
# Print the imbalanced classification report
print(classification_report_imbalanced(y_test, y_pred))

                   pre       rec       spe        f1       geo       iba       sup

          0       1.00      1.00      1.00      1.00      1.00      1.00       101
          1       1.00      1.00      1.00      1.00      1.00      1.00     17104

avg / total       1.00      1.00      1.00      1.00      1.00      1.00     17205



### Final Questions

1. Which model had the best balanced accuracy score?

    The Random Forest Classifier had the better balanced accuracy score of 1.o.

2. Which model had the best recall score?

    Both models had a 1.0 recall.

3. Which model had the best geometric mean score?

    Both models had geometric mean scores of 1.0.

4. What are the top three features?

    The top three features are loan_status, total_rec_prncp, last_pymnt_amnt.